In [ ]:
import yaml
from omegaconf import OmegaConf

import torch
from utils import make_model, set_random_seed, save_model, load_model
from trainer import train
from dataset import ShapeDataset, load_data
from dataset_config import DATASET_CONFIG

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import torchvision.transforms as transforms

import torch.nn.functional as F

from sklearn.cluster import KMeans
import fastcluster
from scipy.cluster.hierarchy import fcluster

import math

import matplotlib.pyplot as plt
from plotting import plot_phases, plot_results, plot_eval, plot_fourier, plot_phases2, plot_masks, plot_slots, build_color_mask, plot_clusters, plot_clusters2


import os
import numpy as np
import imageio
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import display
import ipywidgets as widgets

import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML

import matplotlib.gridspec as gridspec

import seaborn as sns

In [ ]:
sns.set()

# Data Paths

In [ ]:
# Function to load a YAML file
def load_yaml_file(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)['params']

folders = [
    "ccn8/new_tetronimoes/conv_recurrent2/5/linear_lstm_20iters",
    "ccn8/new_tetronimoes/cornn_model2/9/linear_100iters",
    "ccn17/multi_mnist/cornn_model2/11/linear_smaller4_0-100iters_16hc",
]
folder = 'experiments'
hydra_config_file = '.hydra/config.yaml'
paths = [f"{folder}/{curr}" for curr in folders]

configs = [load_yaml_file(f"{p}/{hydra_config_file}") for p in paths]

In [ ]:
# Setup
seed = 1
set_random_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models

In [ ]:
def load_model(cp_folder, config, device, data_config):
    net = make_model(
        device,
        config['model_type'],
        config['num_classes'],
        config['N'],
        config['dt'],
        config['min_iters'],
        config['max_iters'],
        data_config['channels'],
        config['c_mid'],
        config['hidden_channels'],
        config['rnn_kernel'],
        data_config['img_size'],
        config['kernel_init'],
        cell_type=config['cell_type'],
        num_layers=config['num_layers'],
        readout_type=config['readout_type'],
    )
    net.load_state_dict(torch.load(f"{cp_folder}/cp.pt", 
                                   map_location=torch.device('cpu')), 
                                   strict=False)
    net.eval()
    return net.to(device)

In [ ]:
models = [load_model(paths[i], configs[i], device, DATASET_CONFIG[configs[i]['dataset']]) for i in range(len(paths))]

# Forward

In [ ]:
def linear_readout(net, y_seq, B, H, W):
    y_seq = y_seq.reshape(B, net.T, net.c_out, -1)
    y_seq = y_seq.transpose(1, 3)
    fft_vals = net.fc_time(y_seq)
    fft_mag = fft_vals.transpose(1, 3) # (B, K, C, H*W)
    fft_mag = fft_mag.reshape(B, fft_mag.size(1), fft_mag.size(2), H, W)
    return fft_mag

# Set up data

In [ ]:
# Load data
data_config1 = DATASET_CONFIG['new_tetronimoes']
data_config2 = DATASET_CONFIG['multi_mnist']
_, valset1, _ = load_data('new_tetronimoes', data_config1)
_, valset2, _ = load_data('multi_mnist', data_config2)

val_loader = DataLoader(valset1, batch_size=16, shuffle=True, drop_last=False)
batch1 = next(iter(val_loader))
val_loader = DataLoader(valset2, batch_size=16, shuffle=True, drop_last=False)
batch2 = next(iter(val_loader))

testsets = {
    'new_tetronimoes' : batch1,
    'multi_mnist' : batch2,
}

In [ ]:
states = []
ffts = []
masks = []
for i, net in enumerate(models):
    config = configs[i]
    dataset = config['dataset']
    batch = testsets[dataset]
    x, x_target = batch
    batch_size = x.size(0)
    x = x.to(device) #torch.Size([16, 2, 3, 40, 40]) 
    logits, y_seq = net(x)
    fft_mag = linear_readout(net.classifier, y_seq, x.size(0), x.size(-2), x.size(-1))
    states.append(y_seq)
    ffts.append(fft_mag)
    masks.append(logits.argmax(dim=1))

In [ ]:
print(states[0].shape, states[1].shape)

In [ ]:
print(ffts[0].shape, ffts[1].shape)

In [ ]:
print(masks[0].shape, masks[1].shape)

# Plot masks

In [ ]:
def plot_masks(masks, title):
    masks = masks.detach().cpu().numpy()
    fig, axes = plt.subplots(1, 16, figsize=(16, 1))
    for i in range(16):
        axes[i].imshow(masks[i])
        axes[i].set_xticks([])
        axes[i].set_yticks([])
    axes[0].set_title(title)
    plt.show()

In [ ]:
for i, net in enumerate(models):
    plot_masks(masks[i], title=configs[i]['model_type'])

In [ ]:
# 2, 4, 5, 7, 8, 9, 10, 11, 12
samples = [1, 2, 3, 4, 5, 6, 7, 8, 9]

# Look at gifs and choose timesteps we want to plot

In [ ]:
def plot_hidden_state_video(y_seq, sample_idx=0, interval=200, fpath=None):
    """
    Given y_seq of shape (T,B,H,W), animate the hidden state for the sample
    `sample_idx` across timesteps T.
    
    - `interval` controls the animation speed (milliseconds between frames).
    - returns: HTML object that, when displayed in Jupyter, shows the animation.
    """
    T, B, H, W = y_seq.shape
    assert 0 <= sample_idx < B, f"sample_idx must be in [0..{B-1}]"
    
    # Subsample to 100 frames if sequence is too long
    if T > 100:
        indices = np.linspace(0, T-1, 100, dtype=int)
        y_seq = y_seq[indices]
        T = 100
    
    # We'll animate frames across t=0..T-1
    #  shape => (T,H,W)
    y_seq_np = y_seq[:, sample_idx].cpu().numpy()  # -> (T,H,W)
    
    # We can pick vmin/vmax across the entire timeseries for a stable color scale
    vmin = y_seq_np.min()
    vmax = y_seq_np.max()
    
    fig, ax = plt.subplots()
    im = ax.imshow(y_seq_np[0], cmap='bwr', vmin=vmin, vmax=vmax)
    #ax.set_title(f"Hidden state evolution (sample={sample_idx})")
    ax.set_xticks([])
    ax.set_yticks([])
    #fig.tight_layout()
    plt.colorbar(im, ax=ax)
    
    def animate(t):
        im.set_array(y_seq_np[t])
        ax.set_xlabel(f"t = {t}")
        return [im]
    
    ani = animation.FuncAnimation(
        fig, animate, 
        frames=T, 
        interval=interval, 
        blit=True
    )

    if fpath is not None:
        ani.save(f'{fpath}.gif', writer='pillow', fps=5)

    plt.close(fig)  # so that we don't get a duplicate static plot
    #return HTML(ani.to_jshtml())

def plot_hidden(y, sample, channel, interval=200, fpath=None):
    y = torch.transpose(y, 0, 1).detach()
    return plot_hidden_state_video(y[:,:,channel], sample_idx=sample, interval=200, fpath=fpath)

In [ ]:
gif_folder = "gifs"

In [ ]:
for sample in samples:
    fpath = f"{gif_folder}/tetronimoes_lstm_sample-{sample}"
    plot_hidden(states[0], sample=sample, channel=1, fpath=fpath)

In [ ]:
for sample in samples:
    fpath = f"{gif_folder}/tetronimoes_cornn_sample-{sample}"
    plot_hidden(states[1], sample=sample, channel=1, fpath=fpath)

In [ ]:
for sample in samples:
    fpath = f"{gif_folder}/multi_mnist_cornn_sample-{sample}"
    plot_hidden(states[2], sample=sample, channel=1, fpath=fpath)